In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

In [34]:
with open("dataset.txt", "r") as f:
    data = " ".join([line.strip() for line in f])

In [35]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([data])

In [36]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
vector_store = FAISS.from_documents(chunks, embeddings)

In [ ]:
import numpy as np
from umap import UMAP
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

base_vectors = np.array([embeddings.embed_query(chunk.page_content) for chunk in chunks])

query = "Thông tin của Shin"
query_vector = embeddings.embed_query(query)

D, I = vector_store.index.search(np.array([query_vector]), k=10)
similar_vectors = np.array([embeddings.embed_query(chunks[i].page_content) for i in I[0]])

all_vectors = np.vstack([base_vectors, [query_vector], similar_vectors])

umap_2d = UMAP(n_components=2, random_state=42)
vectors_2d = umap_2d.fit_transform(all_vectors)

n_base = len(base_vectors)
labels = ['Base'] * n_base + ['Query'] + ['Similar'] * len(similar_vectors)
colors = {'Base': 'gray', 'Query': 'green', 'Similar': 'red'}

df = pd.DataFrame({
    'UMAP1': vectors_2d[:, 0],
    'UMAP2': vectors_2d[:, 1],
    'Type': labels
})

fig = px.scatter(df, x='UMAP1', y='UMAP2', 
                color='Type',
                color_discrete_map=colors,
                title='Vector Embeddings trong không gian 2D',
                template='plotly_dark')

fig.update_traces(marker=dict(size=8))
fig.update_traces(marker=dict(size=12), selector=dict(name='Query'))
fig.update_traces(marker=dict(size=12), selector=dict(name='Similar'))

fig.show()

fig.write_html("vector_visualization.html", include_plotlyjs=True, full_html=True)

c:\Users\PC\anaconda3\envs\orb-slam-py\lib\site-packages\umap\umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [38]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10})
retriever.invoke('Thông tin của Shin')

[Document(id='e53062ea-f3a3-41de-bfcd-885f7cb92ded', metadata={}, page_content='bị đè sập ý nghĩ và tình thần cô, loáng thoáng nghe được giọng của Shin. “Thiếu tá! Tắt đồng bộ ngay! Thiếu tá Millize!” Trong giọng nói của thiếu niên luôn trầm tĩnh như cậu mang theo vẻ nôn nóng hiếm thấy, nhưng một Lena đã hoảng hốt thì không nghe thấy được. Cô chỉ cố hết sức che tai lại, co rúm người trốn tránh tất cả, gào thét lên mong có thể át đi những tiếng kêu kia, nhưng tiếng kêu lúc lâm chung mãi không ngừng đó dần dần gặm nhấm lý trí cô… “Tch!” Shin chậc lưỡi, tắt đồng bộ đi. Tiếng la hét thảm thiết cũng biến mất ngay. “……A…” Lena chậm chạp ngẩng đầu lên, buông hai tay ra trong sợ hãi… Không nghe thấy gì nữa. Coi bộ tất cả Processor đều đã tắt đồng bộ. Lena không biết đã từ ghế ngã xuống đất từ lúc nào, nhịp thở trở nên dồn dập vì sợ hãi, đôi mắt trợn trừng trong kinh hãi nhìn vào bóng đen trong phòng quản lí. …Thứ khi nãy… là gì…? Đó không phải bắt nguồn từ các Processor cô đồng bộ. Không phải 

In [39]:
from groq import Groq
from apikey import api

client = Groq(api_key=api)


def get_answer_from_context(question: str, context: str) -> str:
    prompt = f"""Câu hỏi: {question}
Ngữ cảnh: {context}
Trả lời:"""
    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {"role": "system", "content": "Bạn là một trợ lý hữu ích. Chỉ trả lời dựa trên dữ liệu cung cấp. Nếu dữ liệu không đủ, chỉ cần nói bạn không biết. Câu trả lời phải luôn là tiếng Việt."},
            {"role": "user", "content": prompt}
        ],
        temperature=1,
        max_tokens=8192,
        top_p=1,
        stream=False
    )
    return response.choices[0].message.content

In [40]:
question = "Shin là ai? Cho tôi biết về thông tin của Shin."
retrieved_docs = retriever.invoke(question)

context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)

answer = get_answer_from_context(question, context_text)
print(answer)

Shin là một nhân vật trong truyện, anh là một Processor, được eğit đào tạo để chiến đấu chống lại các Legion, là các robot được phát triển để giết người. Shin có khả năng chiến đấu vượt trội và được gọi là "Thần chết" do khả năng quyết định thứ tự chết của các chiến binh.

Shin có một quá khứ đau buồn, anh bị cha xứ kéo đi sau khi anh trai của anh tức giận và ra đi. Shin sau đó đã chiến đấu chống lại Legion và trở thành một Processor.

Shin có một vấn đề về trí nhớ, anh không nhớ nhiều chuyện của anh trai và cha mẹ anh. Tuy nhiên, khi nói chuyện với Lena, Shin đã nhớ lại một số kỷ niệm về quá khứ, bao gồm cả một kỷ niệm về pháo hoa tại dinh tổng thống ở khu 1.

Shin có một tính cách lạnh lùng và bình tĩnh, anh ít khi bày tỏ cảm xúc và thường nói lời lẽ lạnh lùng. Tuy nhiên, trong một số tình huống, anh lại thể hiện được một số cảm xúc, chẳng hạn như khó chịu và tức giận.
